# Data Environments

## Overview

Each region defines a data environment which controls how variables are made available to executing threads ([OpenMP 5.1 - 2.21](https://www.openmp.org/spec-html/5.1/openmpse29.html)).
\
Variables can be

|                 |                                                                         |                                                                                |
|-----------------|-------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| `shared`        | all threads access the same variable                                    | [OpenMP 5.1 - 2.21.4.2](https://www.openmp.org/spec-html/5.1/openmpsu116.html) |
| `private`       | each thread holds its own version of the variable in the current region | [OpenMP 5.1 - 2.21.4.3](https://www.openmp.org/spec-html/5.1/openmpsu116.html) |
| `threadprivate` | each thread holds its own version of the variable across regions        | [OpenMP 5.1 - 2.21.2](https://www.openmp.org/spec-html/5.1/openmpsu114.html)   |

Let's first load our ICE magic extension and investigate the effects of *shared* and *private*.

In [ ]:
%load_ext ice_magic

### Shared

In [ ]:
%%cpp_omp -o code/data-env/shared.cpp -e OMP_NUM_THREADS=3

int sharedVar = 20;
#pragma omp parallel shared(sharedVar)
    std::cout << sharedVar++ << std::endl; // race condition

std::cout << sharedVar++ << std::endl;

<div class="alert alert-block alert-warning"> <b>Warning:</b> don't do this -- it introduces a race condition </div>

### Private

In [ ]:
%%cpp_omp -o code/data-env/private.cpp -e OMP_NUM_THREADS=3

int privateVar = 10;
#pragma omp parallel private(privateVar)
    std::cout << privateVar++ << std::endl;

std::cout << privateVar << std::endl;

Private variables are uninitialized and cleaned up at the end of the data environment.
\
Marking variables `firstprivate` copies in the value from the original variable when *entering* the region ([OpenMP 5.1 - 2.21.4.4](https://www.openmp.org/spec-html/5.1/openmpsu116.html)).
\
For loops, `lastprivate` copies the value of the last iteration to the original variable when *exiting* the region (see the lastprivate section in the [loops notebook](loops.ipynb#lastprivate)).

In [ ]:
%%cpp_omp -o code/data-env/firstprivate.cpp -e OMP_NUM_THREADS=3

int privateVar = 10;
#pragma omp parallel firstprivate(privateVar)
    std::cout << ++privateVar << std::endl;

std::cout << privateVar << std::endl;

## Default Behavior

The default behavior follows specific data-sharing attribute rules ([OpenMP 5.1 - 2.21.1](https://www.openmp.org/spec-html/5.1/openmpsu113.html)).
\
Relevant in this scope is that most variables are *shared* by default if not specified otherwise.
Exceptions are
* non-static variables declared in the construct,
* non-static variables declared in called functions, and
* iteration variables of [worksharing loops](loops.ipynb).

### Variables from the Enclosing Scope

In [ ]:
%%cpp_omp -o code/data-env/def-behavior.cpp -e OMP_NUM_THREADS=3

int someVar = 30;
#pragma omp parallel
    // bad practice -- race condition
    std::cout << ++someVar << std::endl;

std::cout << someVar << std::endl;

### Static Variables from the Enclosing Scope

In [ ]:
%%cpp_omp -o code/data-env/def-behavior-static.cpp -e OMP_NUM_THREADS=3

static int staticVar = 30;
#pragma omp parallel
    // bad practice -- race condition
    std::cout << ++staticVar << std::endl;

std::cout << staticVar << std::endl;

### Local Variables

In [ ]:
%%cpp_omp -o code/data-env/def-behavior-local.cpp -e OMP_NUM_THREADS=3

int someVar = 30;
#pragma omp parallel
{
    int localVar = omp_get_thread_num();
    std::cout << ++localVar << std::endl;
}

std::cout << someVar << std::endl;

### Static Local Variables

In [ ]:
%%cpp_omp -o code/data-env/def-behavior-local-static.cpp -e OMP_NUM_THREADS=3

int someVar = 30;
#pragma omp parallel
{
    // bad practice -- race condition
    static int localVar = omp_get_thread_num();
    std::cout << ++localVar << std::endl;
}

std::cout << someVar << std::endl;

### Default Clause

Defaults can be changed with `default(_)` where `_` is one of `shared`, `none`, and since OpenMP v5.0 also `private` and `firstprivate` ([OpenMP 5.1 - 2.21.4.1](https://www.openmp.org/spec-html/5.1/openmpsu116.html)).

<div class="alert alert-block alert-info"> <b>Note:</b> <code>default(none)</code> is recommended for catching errors early </div>

In [ ]:
%%cpp_omp -o code/data-env/default.cpp -e OMP_NUM_THREADS=3

int someVar = 30;
#pragma omp parallel default(none) //firstprivate(someVar) shared(std::cout)
    std::cout << ++someVar << std::endl;

std::cout << someVar << std::endl;